In [ ]:
import json
from queue import Queue

In [ ]:
with open("intermediate_outputs/formatted_appState_w_dependencies.json", "r") as f:
    data = json.load(f)

In [ ]:
queries = {}

for entry in [
    query 
    for query 
    in [{"id": item.get("id"), "query": item["template"].get("query")} for item in data.values()]
    if query["query"] is not None and query["query"] != "" and "SELECT" in query["query"]
]:
    queries[entry["id"]] = entry["query"]

with open("debug_files/queries.txt", 'w') as f:
    f.write(json.dumps(queries, indent=4))
list(queries.values())

In [ ]:
def flatten_list(nested_list):
    return [
        item 
        for sublist 
        in nested_list 
            for item 
            in (
                flatten_list(sublist) 
                if isinstance(sublist, list) 
                else [sublist]
            )
    ]

In [ ]:
def get_tables_from_sql_query(query: str):
    # print("query: ", query)

    if (query is None or query.strip() == ""):
        # print("    --  NONE")
        return [None]

    if ("SELECT" not in query or "FROM" not in query):
        # print("    --  NO SELECT + FROM")
        return [query.split(" ")[0].strip()]

    ret =  flatten_list([
        get_tables_from_sql_query(table.strip())
        for joined 
        in "FROM".join(query.split("FROM")[1:]).split("WHERE")[0].strip().split(",") 
            for table 
            in joined.strip().split("JOIN")
    ])
    # print("    -- ", ret)
    return ret


stripped = {}
for (id, value) in [
    (key,
    {
        "columns": [
            column.strip() #.split("AS")[0].strip()
            for column
            in value.split("SELECT")[1].split("FROM")[0].strip().split(",")
        ],
        "tables": get_tables_from_sql_query(value)
    })
    for key, value 
    in queries.items()
]:
    stripped[id] = value
    tables_str = ", ".join(stripped[id]["tables"])
    if "{{" in queries[id]:
        stripped[id]["dependencies"] = [
            "{{" + dependency.strip().split("}}")[0].strip() + "}}"
            for dependency
            in queries[id].split("{{")[1:]
            if dependency.strip().split("}}")[0].strip()
            not in tables_str
        ]
    else:
        stripped[id]["dependencies"] = []

stripped

# build DOT graph

In [ ]:
# build dot digraph
dot = """digraph sql_SELECT_queries {\n
  node [shape=oval, style=filled]
  edge [color=black]
"""

columns = list(set(flatten_list([value["columns"] for value in stripped.values()])))
tables = list(set(flatten_list([value["tables"] for value in stripped.values()])))
dependencies = list(set(flatten_list([value["dependencies"] for value in stripped.values()])))

nodes = {}

for [index, name] in enumerate(list(set(
    [id for id in stripped.keys()] + 
    columns + \
    tables + \
    dependencies
))):
    nodes[name] = index

for (label, index) in nodes.items():
    fillcolor = "white"
    if label in stripped.keys():
        fillcolor = "#BBD686"
    elif label in dependencies:
        fillcolor = "#D6D6D6"
    elif label in tables:
        fillcolor = "orange"
    dot += f'\t{index} [label="{label}" fillcolor="{fillcolor}"];\n'

for (id, value) in stripped.items():
    dot += f'\tsubgraph columns_{id} ' + '{\n'
    for column in value["columns"]:
        dot += f'\t\t{nodes.get(id)} -> {nodes.get(column)} [ color="gray70" ];\n'
    dot += "\t}\n"
    for table in value["tables"]:
        dot += f'\t{nodes.get(table)} -> {nodes.get(id)} [ color="red" ];\n'
    for dependency in value["dependencies"]:
        dot += f'\t{nodes.get(dependency)} -> {nodes.get(id)};\n'
dot += "}"

with open("outputs/sql_query_columns.dot", 'w') as f:
    f.write(dot)

In [ ]:
# # maximum recursion depth exceeded.
# # -- probably because of an infinite loop

# def build_linked_path(name: str):
#     global data;
#     touched = []
#     linked_path = []
#     q = Queue()
#     q.put(name)

#     while not q.empty():
#         current_name = q.get()
#         if not current_name in touched and current_name in data:
            


In [ ]:
# def build_linked_path(name: str):
#     global data;
#     if name not in data:
#         return {
#             "name": name,
#             "type": None,
#             "chained_events": []
#         }
#     return {
#         "name": name,
#         "type": data[name]["type"],
#         "component_relations": data[name]["component_relations"],
#         "query": data[name].get("query"),
#         "chained_events": [build_linked_path(child["pluginId"]) for child in data[name]["component_relations"].get("dependants", [])]
#     }

# def get_recursive_name(linked_path_node: dict):
#     return {
#         "name": linked_path_node["name"],
#         "chained_events": get_recursive_name(linked_path_node["chained_events"])
#     }

In [ ]:
# # get first key in data
# first_key = list(data.keys())[0]
# first_key

In [ ]:
# linked_path = build_linked_path(first_key)
# names = get_recursive_name(linked_path)